# VESTA FRAUD DETECTION WITH AUTOML - EVALML

**Installation & Importaion of Libraries**

In [ ]:
!pip install "featuretools[complete]"
!pip install composeml
!pip install utils
%pip install evalml
!pip install dask[complete] distributed --upgrade
%pip install -U tornado
!pip install woodwork
!pip install scikit-multilearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import utils
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# featuretools and compose-ML
import composeml as cp
import featuretools as ft
import woodwork as ww

import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types
from evalml.preprocessing import split_data

In [ ]:
# basic machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_score

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.metrics import r2_score

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import FraudCost
from evalml.objectives import CostBenefitMatrix
from evalml import AutoMLSearch
evalml.problem_types.problem_types.ProblemTypes.all_problem_types

from skmultilearn.model_selection import iterative_train_test_split

# Reading Preprocessed Files and Lable encoding
Note: original test file does not have isFraud column.





In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/train_v3.csv')

In [ ]:
##df_test = pd.read_csv('/content/drive/MyDrive/Vesta_Fraud/test_v2.csv')

In [ ]:
df_train

**One Hot Encode**

In [ ]:
prodenc = LabelEncoder()

In [ ]:
df_train['ProductCD'] = prodenc.fit_transform(df_train['ProductCD'])

In [ ]:
df_train.head(5)

In [ ]:
#before one hot encoding and converting to categorical values for card 4
card4enc = LabelEncoder()

In [ ]:
df_train['card4'] = card4enc.fit_transform(df_train['card4'])

In [ ]:
emailenc = LabelEncoder()

In [ ]:
df_train['P_emaildomain'] = emailenc.fit_transform(df_train['P_emaildomain'])

In [ ]:
df_train

In [ ]:
card6enc = LabelEncoder()

In [ ]:
df_train['card6'] = card6enc.fit_transform(df_train['card6'])

In [ ]:
df_train['DeviceType'].value_counts()

In [ ]:
devicetype_enc= LabelEncoder()

In [ ]:
df_train['DeviceType'] =devicetype_enc.fit_transform(df_train['DeviceType'])

In [ ]:
df_train

In [ ]:
deviceinfo_enc = LabelEncoder()

In [ ]:
df_train['DeviceInfo'] =deviceinfo_enc.fit_transform(df_train['DeviceInfo'])

In [ ]:
df_train

In [ ]:
df_train.columns.to_list()

In [ ]:
df_list = df.columns.to_list()

In [ ]:
df_list

##Splitting Data with All Columns##



In [ ]:
# data features - independent variables
X = df_train.loc[:, df_train.columns != 'isFraud']

In [ ]:
pd.set_option('max_columns', None)

In [ ]:
X

In [ ]:
# data lables - target to predict - dependent variable
y = df_train.isFraud

##Splitting Data with Feature Importance Columns##

In [ ]:
##columns lables that will predict transaction is fraud
col_features = df_train[['TransactionAmt','TransactionDT','TransactionID','addr1','ProductCD','P_emaildomain','card1',
                          'card2','card3','card4','card5','card6','D1','D15','C1','C11','C12','C13','C14','C2','C4','C6','C7','V15',
                          'V16','V17','V18','V33','V34','V53','V58','V73','V74','V79','V80','V81','V84','V86','V87','V92','V93','V94','V312','V317','V318','DeviceInfo']]

In [ ]:
df_train['V16'].value_counts()

In [ ]:
#Y lables this is what to be predicted for X features
col_labels = df_train['isFraud']

In [ ]:
features = col_features
labels = col_labels

In [ ]:
# variable X are the features
# variable y are the labels
X = features
y = labels

print(features.head())
print(X.shape)

print(labels.head())
print(y.shape)

##Not required to run

In [ ]:
import evalml
from evalml import AutoMLSearch
from evalml.utils import infer_feature_types
from evalml.preprocessing import split_data

In [ ]:
#Fraud Objective
#retry_percentage - what percentage of customers will retry a transaction if it is declined?
#interchange_fee - how much of each successful transaction do you collect?
#fraud_payout_percentage - the percentage of fraud will you be unable to collect
#amount_col - the column in the data the represents the transaction amount

from evalml.objectives import FraudCost
fraud_objective = FraudCost(retry_percentage=.4,
                            interchange_fee=.03,
                            fraud_payout_percentage=.65,
                            amount_col='TransactionAmt')

In [ ]:
#autosearch and optimized for fraud objective
automl = AutoMLSearch(X_train = X_train, y_train = y_train,
                      problem_type='binary',
                      objective= fraud_objective,
                      additional_objectives=['auc', 'f1', 'precision','Accuracy Binary'],
                      allowed_model_families=["lightgbm","xgboost","random_forest", "decision_tree","linear_model"],
                      max_batches =1, 
                      sampler_balanced_ratio = 0.75,
                      optimize_thresholds= True,
                      verbose = True)

In [ ]:
#autosearch and optimized for AUC objective
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='binary',
                          objective='auc',
                          additional_objectives=['f1', 'precision'],
                          max_batches=1,
                          sampler_balanced_ratio = 0.75,
                          allowed_model_families=["random_forest", "decision_tree", "lightgbm", "xgboost","linear_model"],
                          optimize_thresholds=True,
                          verbose = True)

#MODELLING#

##ADDING FRAUD OBJECTIVE##

Example of fraud objective

https://evalml.alteryx.com/en/v0.11.0/demos/fraud.html

https://evalml.alteryx.com/en/stable/demos/fraud.html

To execute fraud objectives , X cannot be scaled. If not, it will throw an error.

In [ ]:
#retry_percentage - what percentage of customers will retry a transaction if it is declined?
#interchange_fee - how much of each successful transaction do you collect?
#fraud_payout_percentage - the percentage of fraud will you be unable to collect
#amount_col - the column in the data the represents the transaction amount

from evalml.objectives import FraudCost
fraud_objective = FraudCost(retry_percentage=.4,
                            interchange_fee=.03,
                            fraud_payout_percentage=.65,
                            amount_col='TransactionAmt')

In [ ]:
# split data into training and testing data sets
# holdout is also known as testing data
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=0.2)

In [ ]:
X_train

In [ ]:
X_holdout

##Confusion Matrix function##

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score
def plot_confusion_matrix(y_test, y_pred):
    acc = round(accuracy_score(y_test, y_pred), 2)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt=".0f")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Accuracy Score: {0}'.format(acc), size=10)
    plt.show()


##Handling Imbalanced Data##


In [ ]:
# Undersampler case
# we don't want to undersample this class, so class 1 (minority) will have a ratio of 1 to itself
# for the majority class 0, we want to undersample it to have a minority/majority ratio of 0.5, 
#which means we want majority to have 2x the samples as the minority
#sampler_ratio_dict = {0: 0.5, 1: 1}

In [ ]:
#pipeline_parameters = {"Undersampler": {"sampler_balanced_ratio": sampler_ratio_dict}}

##use autoML to search for the best model by specifying model families with fraud objective ##

In [ ]:
#autosearch with fraud objective
automl = AutoMLSearch(X_train = X_train, y_train = y_train,
                      problem_type='binary',
                      objective= fraud_objective,
                      additional_objectives=['auc', 'f1', 'precision','Accuracy Binary'],
                      allowed_model_families=["lightgbm","xgboost","random_forest", "decision_tree","linear_model"],
                      max_batches =1, 
                      sampler_balanced_ratio = 0.75,
                      optimize_thresholds= True,
                      verbose = True)

In [ ]:
automl.allowed_pipelines[-1]

In [ ]:
automl.search()

In [ ]:
automl.rankings

In [ ]:
automl.describe_pipeline(automl.rankings.iloc[2]["id"])

In [ ]:
best_pipeline = automl.best_pipeline

In [ ]:
best_pipeline.graph()

In [ ]:
best_pipeline.score(X_train, y_train, objectives=["auc", fraud_objective])


In [ ]:
best_pipeline.score(X_train, y_train,['Accuracy Binary', 'F1', 'AUC', 'recall', fraud_objective])

In [ ]:
best_pipeline.score(X_holdout, y_holdout,['Accuracy Binary', 'F1', 'AUC', 'recall', fraud_objective])

In [ ]:
print(best_pipeline.threshold)

In [ ]:
y_pred_proba = best_pipeline.predict_proba(X_train)

In [ ]:
print(y_pred_proba)

##Avoiding Overfitting##

**Detecting lable leakage**
TargetLeakageDataCheck checks for features that could potentially be “leaking”
information by calculating the Pearson correlation coefficient between each feature and the target to warn users if there
are features are highly correlated with the target.
https://evalml.alteryx.com/_/downloads/en/stable/pdf/

** How does EvalML avoid overfitting?
EvalML provides data checks to combat overfitting. Such data checks include detecting label leakage, unstable
pipelines, hold-out datasets and cross validation. EvalML defaults to using Stratified K-Fold cross-validation for
classification problems and K-Fold cross-validation for regression problems but allows you to utilize your own crossvalidation methods as well.**

In [ ]:
from evalml.data_checks import TargetLeakageDataCheck 

In [ ]:
Target_leakage_check = TargetLeakageDataCheck()
results = Target_leakage_check.validate(X_train, y_train)

for message in results['warnings']:print("Warning:", message['message'])
for message in results['errors']: print("Error:", message['message'])

##use autoML to search for the best model by specifying 4 model families (AUC objective)##

In [ ]:
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='binary',
                          objective='auc',
                          additional_objectives=['f1', 'precision'],
                          max_batches=1,
                          sampler_balanced_ratio = 0.75,
                          allowed_model_families=["random_forest", "decision_tree", "lightgbm", "xgboost","linear_model"],
                          optimize_thresholds=True,
                          verbose = True)

In [ ]:
automl_auc.allowed_pipelines[-1]

In [ ]:
automl_auc.search()

In [ ]:
automl_auc.rankings

In [ ]:
best_pipeline_auc = automl_auc.best_pipeline

In [ ]:
best_pipeline_auc.graph()

In [ ]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", fraud_objective])

In [ ]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_train, y_train,  objectives=["auc", fraud_objective])

In [ ]:
# fraud score on fraud optimized again. Found that fraud cost is lower compared to above.
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

In [ ]:
best_pipeline_auc.score(X_holdout, y_holdout,['Accuracy Binary', 'F1', 'AUC', 'recall', fraud_objective])

In [ ]:
print(best_pipeline_auc.threshold)

#VISUALISATION AND EVALUATION#

##Predicting with best pipeline generated from fraud objective##

In [ ]:
y_pred = best_pipeline.predict(X_holdout)

In [ ]:
from evalml.model_understanding.graphs import graph_confusion_matrix

**Note: Many false positives.**

In [ ]:
#graph_confusion_matrix(y_holdout, y_pred, normalize_method= None)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred)

In [ ]:
best_pipeline.graph_feature_importance()

##Predicting with best pipeline generated from auc metric##

**Note: many false negatives**

In [ ]:
from evalml.model_understanding.graphs import graph_confusion_matrix

In [ ]:
y_pred_auc = best_pipeline_auc.predict(X_holdout)

In [ ]:
#graph_confusion_matrix(y_holdout, y_pred_2, normalize_method= None)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_auc)

In [ ]:
best_pipeline_auc.graph_feature_importance()

##Training and Scoring Multiple 4 Pipelines generated with Fraud Objective##

In [ ]:
trained_pipelines = automl.train_pipelines([automl.get_pipeline(i) for i in [1, 2,3, 4,5,6]])

In [ ]:
trained_pipelines

In [ ]:
pipeline_train_scores = automl.score_pipelines([trained_pipelines[name] for name in trained_pipelines.keys()],
                                                X_train,
                                                y_train,
                                                ['Accuracy Binary', 'F1', 'AUC', 'recall',fraud_objective])

In [ ]:
pipeline_train_scores

In [ ]:
pipeline_test_scores = automl.score_pipelines([trained_pipelines[name] for name in trained_pipelines.keys()],
                                                X_holdout,
                                                y_holdout,
                                                ['Accuracy Binary', 'F1', 'AUC', 'recall',fraud_objective])

In [ ]:
pipeline_test_scores

**manual fitting with other pipelines to compare (with Fraud objective)**

**fitting with first best pipeline - Decision Tree**

In [ ]:
first_best_pipeline = automl.get_pipeline(2)

In [ ]:
first_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_1 = first_best_pipeline.predict(X_holdout, fraud_objective)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_1)

**fitting with second best pipeline - Elastic Net Classifier**

In [ ]:
second_best_pipeline = automl.get_pipeline(1)

In [ ]:
second_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_2 = second_best_pipeline.predict(X_holdout, fraud_objective)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_2)

In [ ]:
##graph_confusion_matrix(y_holdout, y_pred_3, normalize_method= None)

In [ ]:
y_pred_proba = second_best_pipeline.predict_proba(X_train)

In [ ]:

#doesnt work
#pipeline.threshold = fraud_objective.optimize_threshold(y_pred_proba, y_train)

In [ ]:
print(second_best_pipeline.threshold)

**fitting with third best pipeline - Logistic Regression**

In [ ]:
third_best_pipeline = automl.get_pipeline(4)

In [ ]:
third_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_3 = third_best_pipeline.predict(X_holdout)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_3)

In [ ]:
#graph_confusion_matrix(y_holdout, y_pred_4, normalize_method= None)

**fitting with fourth best pipeline - Random Forest**

In [ ]:
fourth_best_pipeline = automl.get_pipeline(6)

In [ ]:
fourth_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_4 = fourth_best_pipeline.predict(X_holdout)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_4)

**fitting with fifth best pipeline - XGBoost**

In [ ]:
fifth_best_pipeline = automl.get_pipeline(3)

In [ ]:
fifth_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_5 = fifth_best_pipeline.predict(X_holdout)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_5)

**fitting with sixth best pipeline - LightGBM**

In [ ]:
sixth_best_pipeline = automl.get_pipeline(5)

In [ ]:
sixth_best_pipeline.fit(X_train,y_train)

In [ ]:
y_pred_6 = sixth_best_pipeline.predict(X_holdout)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_6)



##Precision Recall Curve Using EvalML for BEST PIPELINES##

**Precision recall curve for best pipeline with fraud objective**

In [ ]:
from evalml.model_understanding.graphs import graph_precision_recall_curve

In [ ]:
#Decision Tree
y_pred_proba = best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba)

**Precision recall curve for best pipeline with auc objective**

In [ ]:
y_pred_proba = best_pipeline_auc.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba)

##Precision recall curve for other pipelines with fraud objective##

In [ ]:
#ElasticNet
y_pred_proba_2 = second_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba_2)

In [ ]:
#LogisticRegression
y_pred_proba_3 = third_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba_3)

In [ ]:
#RandomForest
y_pred_proba_4 = fourth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba_4)

In [ ]:
#XGBoost
y_pred_proba_5 = fifth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba_5)

In [ ]:
#LightGBM
y_pred_proba_6 = sixth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_precision_recall_curve(y_holdout, y_pred_proba_6)

##ROC Curve using EvalML for BEST PIPELINES##

**ROC curve for best pipeline with fraud objective**

In [ ]:
from evalml.model_understanding.graphs import graph_roc_curve

In [ ]:
#Decision Tree
y_pred_proba_1 = best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_1)

**ROC curve for best pipeline with auc objective**

In [ ]:
#LightGBM
y_pred_proba_auc = best_pipeline_auc.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_auc)

##ROC curve for other pipelines with fraud objective##

In [ ]:
#DecisionTree
y_pred_proba_1_1 = first_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_1_1)

In [ ]:
#ElasticNet
y_pred_proba_2 = second_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_2)

In [ ]:
#LogisticRegression
y_pred_proba_3 = third_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_3)

In [ ]:
#RandomForest
y_pred_proba_4 = fourth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_4)

In [ ]:
#XGBoost
y_pred_proba_5 = fifth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_5)

In [ ]:
#LightGBM
y_pred_proba_6 = sixth_best_pipeline.predict_proba(X_holdout)

In [ ]:
graph_roc_curve(y_holdout, y_pred_proba_6)

##FEATURE IMPORTANCE for all pipelines generated with Fraud Objective##

**auto-best pipeline**

In [ ]:
best_pipeline.graph_feature_importance()

**manual-fit best pipeline**

In [ ]:
first_best_pipeline.graph_feature_importance()

**second best pipeline**

In [ ]:
#ElasticNet
second_best_pipeline.graph_feature_importance()

**third best pipeline**

In [ ]:
#LogisticRegression
third_best_pipeline.graph_feature_importance()

**fourth pipleline**

In [ ]:
#RandomForest
fourth_best_pipeline.graph_feature_importance()

In [ ]:
#XGBoost
fifth_best_pipeline.graph_feature_importance()

In [ ]:
#LightGBM
sixth_best_pipeline.graph_feature_importance()

##Training and Scoring Multiple Pipelines generated with AUC objective##

In [ ]:
trained_pipelines_auc = automl_auc.train_pipelines([automl_auc.get_pipeline(i) for i in [0, 1, 2,3, 4]])

In [ ]:
pipeline_holdout_scores_auc = automl_auc.score_pipelines([trained_pipelines_auc[name] for name in trained_pipelines_auc.keys()],
                                                X_holdout,
                                                y_holdout,
                                                ['Accuracy Binary', 'F1', 'AUC'])

In [ ]:
pipeline_holdout_scores_auc

**manual fitting with first best pipeline instead of using best_pipeline default function to compare**

In [ ]:
first_best_pipeline_auc =  automl_auc.get_pipeline(4)

In [ ]:
first_best_pipeline_auc

In [ ]:
first_best_pipeline_auc.fit(X_train,y_train)

In [ ]:
y_pred_7 = first_best_pipeline_auc.predict(X_holdout)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred_7)

In [ ]:
#graph_confusion_matrix(y_holdout, y_pred_7, normalize_method= None)

In [ ]:
from evalml.pipelines import BinaryClassificationPipeline

In [ ]:
pipeline_binary = BinaryClassificationPipeline(['Simple Imputer', 'Random Forest Classifier'])

In [ ]:
from evalml.model_understanding.graphs import confusion_matrix
y_pred = pipeline_binary.predict(X_holdout)
confusion_matrix(y_holdout, y_pred)

In [ ]:
from evalml.model_understanding.graphs import graph_confusion_matrix
y_pred = pipeline_binary.predict(X_holdout)
#graph_confusion_matrix(y_holdout, y_pred)

In [ ]:
plot_confusion_matrix(y_holdout, y_pred)